In [2]:
import pandas as pd
import numpy as np

demographic_df = pd.read_csv('https://www.physionet.org/physiobank/database/challenge/2018/age-sex.csv')
demographic_df.describe()

,Age
count,1983.000000
mean,54.988401
std,14.325831
min,18.000000
25%,46.000000
50%,56.000000
75%,65.000000
max,93.000000


In [3]:
len(demographic_df)  # 1983

1983

In [11]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil

import wfdb
# Demo 1 - Read a wfdb record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data.
record = wfdb.rdrecord('https://www.physionet.org/physiobank/database/challenge/2018/training/tr03-0005/') 
wfdb.plot_wfdb(record=record, title='Record a103l from Physionet Challenge 2015') 
display(record.__dict__)


# Can also read the same files hosted on Physiobank https://physionet.org/physiobank/database/
# in the challenge/2015/training/ database subdirectory. Full url = https://physionet.org/physiobank/database/challenge/2015/training/
record2 = wfdb.rdrecord('a103l', pb_dir = 'https://physionet.org/physiobank/database/challenge/2015/training/')

FileNotFoundError: [Errno 2] No such file or directory: 'https://www.physionet.org/physiobank/database/challenge/2018/training/tr03-0005/.hea'

In [ ]:
wfdb.rdann('sample-data/100', 'atr')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

/Users/isabelmetzger/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,Unnamed: 0,id,text,CoverageLevel
0,1,1,@_derricklamar @sly9297 has a root canal sched...,High
1,2,2,In such horrible dental pain. Have good insura...,High
2,3,3,omg I am in sooo much fucking pain. I have a d...,High
3,4,4,"""@leahhfragg @RandeeeGE LMFAOOO ""I can't affor...",High
4,5,5,I do not want to go to the dentist tomorrow. H...,High


,text,label
0,@_derricklamar @sly9297 has a root canal sched...,1
1,In such horrible dental pain. Have good insura...,1
2,omg I am in sooo much fucking pain. I have a d...,1
3,"""@leahhfragg @RandeeeGE LMFAOOO ""I can't affor...",1
4,I do not want to go to the dentist tomorrow. H...,1


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 2 columns):
text     1502 non-null object
label    1502 non-null int64
dtypes: int64(1), object(1)
memory usage: 23.5+ KB


In [82]:
X = df.text
y = df.label
le = LabelEncoder()
Y = le.fit_transform(y)
Y = y.reshape(-1,1)
print(Y[0:3])

[[1]
 [1]
 [1]]


/Users/isabelmetzger/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [87]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)
X_train.head()

520     gonna see about maybe getting something less s...
1228    @Horsewhispers : Oh believe me. I pop NSAIDs l...
1246    @DemonikaDevour sorry, i need a dentist but a ...
1464    And if it is wisdom teeth, then I'm fucked. No...
1468    You know what sucks? Having like 3 cavities bu...
Name: text, dtype: object

In [88]:
max_words = 5000
max_len = 500
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [89]:
import keras.backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))


def categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)))


def sparse_categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())))


def top_k_categorical_accuracy(y_true, y_pred, k=5):
    return K.mean(K.in_top_k(y_pred, K.argmax(y_true, axis=-1), k))


def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def mean_absolute_percentage_error(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None))
    return 100. * K.mean(diff)


def mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.mean(K.square(first_log - second_log))


def hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.))


def squared_hinge(y_true, y_pred):
    return K.mean(K.square(K.maximum(1. - y_true * y_pred, 0.)))


def categorical_crossentropy(y_true, y_pred):
    return K.mean(K.categorical_crossentropy(y_pred, y_true))


def sparse_categorical_crossentropy(y_true, y_pred):
    return K.mean(K.sparse_categorical_crossentropy(y_pred, y_true))


def binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_pred, y_true))


def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.mean(K.sum(y_true * K.log(y_true / y_pred), axis=-1))


def poisson(y_true, y_pred):
    return K.mean(y_pred - y_true * K.log(y_pred + K.epsilon()))


def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.mean(y_true * y_pred)


def matthews_correlation(y_true, y_pred):
    """Matthews correlation metric.
    It is only computed as a batch-wise average, not globally.
    Computes the Matthews correlation coefficient measure for quality
    of binary classification problems.
    """
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)


# aliases
mse = MSE = mean_squared_error
mae = MAE = mean_absolute_error
mape = MAPE = mean_absolute_percentage_error
msle = MSLE = mean_squared_logarithmic_error
cosine = cosine_proximity
fscore = f1score = fmeasure

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', recall, precision, fmeasure])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 500)               0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 500, 100)          500000    
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_22 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [90]:
Y_train[0:10]

520     0
1228    1
1246    1
1464    1
1468    1
711     1
763     0
824     1
822     1
166     1
Name: label, dtype: int64

In [91]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=3,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

Train on 960 samples, validate on 241 samples
Epoch 1/3
960/960 [==============================] - 6s 7ms/step - loss: 0.6635 - acc: 0.6490 - recall: 0.9302 - precision: 0.6692 - fmeasure: 0.7722 - val_loss: 0.6450 - val_acc: 0.6598 - val_recall: 1.0000 - val_precision: 0.6598 - val_fmeasure: 0.7948
Epoch 2/3
960/960 [==============================] - 4s 5ms/step - loss: 0.6210 - acc: 0.6771 - recall: 1.0000 - precision: 0.6771 - fmeasure: 0.8073 - val_loss: 0.6430 - val_acc: 0.6598 - val_recall: 1.0000 - val_precision: 0.6598 - val_fmeasure: 0.7948
Epoch 3/3
960/960 [==============================] - 4s 4ms/step - loss: 0.6038 - acc: 0.6771 - recall: 1.0000 - precision: 0.6771 - fmeasure: 0.8072 - val_loss: 0.6471 - val_acc: 0.6598 - val_recall: 1.0000 - val_precision: 0.6598 - val_fmeasure: 0.7948


In [92]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Recall: {:0.3f}\n  Precision: {:0.3f}\n  fmeasure: {:0.3f}'.format(accr[0],accr[1],
                                                                                                                            accr[2], accr[3], accr[4]))
accr  #Accuracy: 0.702
from sklearn.metrics import f1_score
predictions = model.predict(test_sequences_matrix,  verbose=0)  #  Accuracy: 0.702
model.metrics_names


301/301 [==============================] - 1s 2ms/step
Test set
  Loss: 0.613
  Accuracy: 0.701
  Recall: 1.000
  Precision: 0.701
  fmeasure: 0.824


['loss', 'acc', 'recall', 'precision', 'fmeasure']

Test set
  Loss: 0.633
  Accuracy: 0.7

In [73]:
# deep

In [93]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000

# truncate and pad input sequences
max_review_length = 500
tok = Tokenizer(num_words=top_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall, precision, fmeasure])
print(model.summary())
model.fit(sequences_matrix,Y_train, epochs=3, batch_size=64)
# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
1201/1201 [==============================] - 6s 5ms/step - loss: 0.6563 - acc: 0.6719 - recall: 0.9878 - precision: 0.678

In [94]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_review_length)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Recall: {:0.3f}\n  Precision: {:0.3f}\n  fmeasure: {:0.3f}'.format(accr[0],accr[1],
                                                                                                                            accr[2], accr[3], accr[4]))
accr  #Accuracy: 0.702
from sklearn.metrics import f1_score
predictions = model.predict(test_sequences_matrix,  verbose=0)  #  Accuracy: 0.702
model.metrics_names

301/301 [==============================] - 1s 4ms/step
Test set
  Loss: 0.610
  Accuracy: 0.701
  Recall: 1.000
  Precision: 0.701
  fmeasure: 0.824


['loss', 'acc', 'recall', 'precision', 'fmeasure']

In [95]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(12)
# load the dataset but only keep the top n words, zero the rest
top_words = 1000

# truncate and pad input sequences
max_review_length = 100
tok = Tokenizer(num_words=top_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall, precision, fmeasure])
print(model.summary())
model.fit(sequences_matrix, Y_train, epochs=3, batch_size=64)
# Final evaluation of the model
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_review_length)
accr = model.evaluate(test_sequences_matrix,Y_test, verbose=0)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Recall: {:0.3f}\n  Precision: {:0.3f}\n  fmeasure: {:0.3f}'.format(accr[0],accr[1],
                                                                                                                            accr[2], accr[3], accr[4]))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 32)           32000     
_________________________________________________________________
lstm_20 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 85,301
Trainable params: 85,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
1201/1201 [==============================] - 4s 3ms/step - loss: 0.6519 - acc: 0.6678 - recall: 0.9784 - precision: 0.6712 - fmeasure: 0.7947
Epoch 2/3
1201/1201 [==============================] - 2s 2ms/step - loss: 0.6333 - acc: 0.6736 - recall: 1.0000 - precision: 0.6736 - fmeasure: 0.8033
Epoch 3/3
1201/1201 [==============================] - 2s 2ms/step - loss: 0.6235 - acc: 0.67

In [96]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

# Embedding
max_features = 20000
max_len = 100
embedding_size = 128
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''

print('Loading data...')



test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=max_len))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',recall, precision, fmeasure])
#, recall, precision, fmeasure]
print('Train...')
model.fit(sequences_matrix, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_sequences_matrix, Y_test))
score, acc,rec, precision, fscore  = model.evaluate(test_sequences_matrix, Y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
print('Test recall:', rec)
print('Test precision:', precision)
print('Test fmeasure:', fscore)


Loading data...
Build model...
Train...
Train on 1201 samples, validate on 301 samples
Epoch 1/2
1201/1201 [==============================] - 5s 5ms/step - loss: 0.6446 - acc: 0.6736 - recall: 1.0000 - precision: 0.6736 - fmeasure: 0.8018 - val_loss: 0.6140 - val_acc: 0.7010 - val_recall: 1.0000 - val_precision: 0.7010 - val_fmeasure: 0.8235
Epoch 2/2
301/301 [==============================] - 0s 243us/step
Test score: 0.6106568771937361
Test accuracy: 0.700996683681526
Test recall: 0.9999999996039558
Test precision: 0.7009966832854819
Test fmeasure: 0.8234838597798269


from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.noise import AlphaDropout
from keras.preprocessing.text import Tokenizer

max_words = 1000
batch_size = 16
epochs = 40
plot = True


def create_network(n_dense=6,
                   dense_units=16,
                   activation='selu',
                   dropout=AlphaDropout,
                   dropout_rate=0.1,
                   kernel_initializer='lecun_normal',
                   optimizer='adam',
                   num_classes=2,
                   max_words=max_words):
    """Generic function to create a fully-connected neural network.
    # Arguments
        n_dense: int > 0. Number of dense layers.
        dense_units: int > 0. Number of dense units per layer.
        dropout: keras.layers.Layer. A dropout layer to apply.
        dropout_rate: 0 <= float <= 1. The rate of dropout.
        kernel_initializer: str. The initializer for the weights.
        optimizer: str/keras.optimizers.Optimizer. The optimizer to use.
        num_classes: int > 0. The number of classes to predict.
        max_words: int > 0. The maximum number of words per data point.
    # Returns
        A Keras model instance (compiled).
    """
    model = Sequential()
    model.add(Dense(dense_units, input_shape=(max_words,),
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model


network1 = {
    'n_dense': 6,
    'dense_units': 16,
    'activation': 'relu',
    'dropout': Dropout,
    'dropout_rate': 0.5,
    'kernel_initializer': 'glorot_uniform',
    'optimizer': 'sgd'
}

network2 = {
    'n_dense': 6,
    'dense_units': 16,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.1,
    'kernel_initializer': 'lecun_normal',
    'optimizer': 'sgd'
}

print('Loading data...')

print(len(sequences_matrix), 'train sequences')
print(len(test_sequences_matrix), 'test sequences')

num_classes = np.max(Y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(X_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

print('\nBuilding network 1...')

model1 = create_network(num_classes=num_classes, **network1)
history_model1 = model1.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.1)

score_model1 = model1.evaluate(x_test,
                               y_test,
                               batch_size=batch_size,
                               verbose=1)


print('\nBuilding network 2...')
model2 = create_network(num_classes=num_classes, **network2)

history_model2 = model2.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.1)

score_model2 = model2.evaluate(x_test,
                               y_test,
                               batch_size=batch_size,
                               verbose=1)

print('\nNetwork 1 results')
print('Hyperparameters:', network1)
print('Test score:', score_model1[0])
print('Test accuracy:', score_model1[1])
print('Network 2 results')
print('Hyperparameters:', network2)
print('Test score:', score_model2[0])
print('Test accuracy:', score_model2[1])

plt.plot(range(epochs),
         history_model1.history['val_loss'],
         'g-',
         label='Network 1 Val Loss')
plt.plot(range(epochs),
         history_model2.history['val_loss'],
         'r-',
         label='Network 2 Val Loss')
plt.plot(range(epochs),
         history_model1.history['loss'],
         'g--',
         label='Network 1 Loss')
plt.plot(range(epochs),
         history_model2.history['loss'],
         'r--',
         label='Network 2 Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('comparison_of_networks.png')